In [9]:
%load_ext autoreload
%autoreload 

import matplotlib
# matplotlib.style.use('fivethirtyeight')
import finplot as fplt

import numpy as np
import pandas as pd
import pandas_bokeh

import polygon_backfill as pb
import market_cluster as mc
import s3_datasets as s3d
import filters as ft
import local_backfill as lb
import bars as sb

symbol='GLD'
date='2020-08-27'
result_path='/Users/bobcolner/QuantClarity/pandas-polygon/data'

# .plot_bokeh(kind='hist', bins=50, sizing_mode="scale_height", disable_scientific_axes=True)
# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
ticks_df = lb.load_ticks(symbol, date, result_path)

1  duplicated trade removed


In [5]:
thresh = {
    'renko': 0.2
}
df = ticks_df[0:10000]
bars, _ = sb.build_bars(df, thresh, as_df=True)

100%|██████████| 9999/9999 [00:02<00:00, 3576.79it/s]


In [10]:
fplt

<module 'finplot' from '/Users/bobcolner/opt/miniconda3/envs/backfill/lib/python3.7/site-packages/finplot/__init__.py'>

In [13]:
import finplot as fplt
import numpy as np
import pandas as pd
import requests

# pull some data
symbol = 'USDT-BTC'
url = 'https://bittrex.com/Api/v2.0/pub/market/GetTicks?marketName=%s&tickInterval=fiveMin' % symbol
data = requests.get(url).json()

# format it in pandas
df = pd.DataFrame(data['result'])
df = df.rename(columns={'T':'time', 'O':'open', 'C':'close', 'H':'high', 'L':'low', 'V':'volume'})
df = df.astype({'time':'datetime64[ns]'})

In [14]:
df

,open,high,low,close,volume,time,BV,marker
0,11611.115600,11615.473300,11586.537000,11586.5370,1.811326,2020-08-05 13:30:00,21002.249466,NaN
1,11585.169100,11585.169100,11564.805003,11577.0890,0.579735,2020-08-05 13:35:00,6709.436485,NaN
2,11578.215915,11586.076800,11573.984000,11585.2770,1.010271,2020-08-05 13:40:00,11697.291772,NaN
3,11575.339400,11584.962000,11562.052000,11578.1940,1.107367,2020-08-05 13:45:00,12811.411889,NaN
4,11584.861998,11595.190000,11581.527999,11592.8643,1.567517,2020-08-05 13:50:00,18167.337507,NaN
...,...,...,...,...,...,...,...,...
11515,10505.343007,10545.637900,10504.368000,10545.6379,1.108965,2020-09-14 13:05:00,11665.651816,NaN
11516,10545.637900,10590.000000,10540.262204,10567.9281,8.468850,2020-09-14 13:10:00,89539.637930,NaN
11517,10575.053600,10589.999998,10546.969600,10546.9696,1.839874,2020-09-14 13:15:00,19447.711097,NaN
11518,10546.969600,10564.142400,10546.869600,10564.1424,0.656360,2020-09-14 13:20:00,6928.619706,NaN


In [13]:
# create two plots
ax,ax2 = fplt.create_plot(symbol, rows=2)

# plot candle sticks
candles = df[['time','open','close','high','low']]
fplt.candlestick_ochl(candles, ax=ax)

# overlay volume on the top plot
volumes = df[['time','open','close','volume']]
fplt.volume_ocv(volumes, ax=ax.overlay())

# put an MA on the close price
fplt.plot(df['time'], df['close'].rolling(25).mean(), ax=ax, legend='ma-25')

# place some dumb markers on low wicks
lo_wicks = df[['open','close']].T.min() - df['low']
df.loc[(lo_wicks>lo_wicks.quantile(0.99)), 'marker'] = df['low']
fplt.plot(df['time'], df['marker'], ax=ax, color='#4a5', style='^', legend='dumb mark')

# draw some random crap on our second plot
fplt.plot(df['time'], np.random.normal(size=len(df)), ax=ax2, color='#927', legend='stuff')
fplt.set_y_range(-1.4, +3.7, ax=ax2) # hard-code y-axis range limitation

# restore view (X-position and zoom) if we ever run this example again
fplt.autoviewrestore()

# we're done
fplt.show()

In [12]:
# create two plots
ax = fplt.create_plot('GLD bars')

# plot candle sticks
candles = df[['open_at', 'price_open', 'price_close', 'price_high', 'price_low']]
fplt.candlestick_ochl(candles, ax=ax)

# overlay volume on the top plot
volumes = df[['open_at', 'price_open', 'price_close', 'volume_sum']]
fplt.volume_ocv(volumes, ax=ax.overlay())



KeyError: "None of [Index(['open_at', 'price_open', 'price_close', 'price_high', 'price_low'], dtype='object')] are in the [columns]"

In [ ]:
fplt.candlestick_ochl(bars[['price_open', 'price_close', 'price_high', 'price_low']])
fplt.show()

In [ ]:
bars.reset_index().price_vwap.plot(figsize=[20,7])

In [ ]:
# add price filters

df = ft.add_filters(df, 'price')

In [ ]:
df[['price', 'filter_jma710']][20000:30000].plot(figsize=[20,8])
# df[['price', 'filter_jma71000']][20000:30000].plot_bokeh(kind='line', sizing_mode="scale_height", disable_scientific_axes=True)

In [ ]:
df2 = df.rename(columns={'sip_dt': 'date_time', 'size': 'volume'})

df2 = df2[['date_time', 'price', 'volume']]

In [ ]:
# get data from s3
clean_df, bad_df = s3d.get_s3_df(symbol, date)

# list_s3_symbol('SPY')
df = s3d.get_s3_df('GLD', '2020-08-27', columns=['exchange_epoch', 'price', 'size', 'condition'])
# df = s3d.get_s3_df(symbol='SPY', date='2020-08-28', columns=['exchange_epoch', 'price', 'size', 'condition'])

# df = pd.read_feather('data.feather', columns=['exchange_epoch', 'price', 'size', 'condition'])
# df['date_time'] = pd.to_datetime(df.exchange_epoch)

In [ ]:
# get local market daily data
mdf = pd.read_parquet('data/mdf.parquet')

sdf = mdf[mdf.symbol=='SPY']

In [ ]:
### market clustering

# %time df = read_market_daily(result_path)
%time mdf = pd.read_parquet('data/mdf.parquet')

%time npdf, sharpe_ratios = mc.normalize_market_df(mdf)

# %time npdf_resid = colwise_linreg(npdf)
%time npdf_resid = pd.read_parquet('data/npdf_resid.parquet')

# %time par_cor_mat = npdf_resid.corr()
%time resid_par_cor_mat = pd.read_parquet('data/partial_cor_mat.parquet')

%time details_df = pd.read_parquet('data/details_df.parquet')

# cluster_lables = cluster_sim_matrix(similarity=resid_par_cor_mat.abs())
%time cluster_lables = pd.read_parquet('data/cluster_lables.parquet')

# symbol_meta = join_symbol_data(details_df, cluster_lables, sharpe_ratios, mdf)
%time symbol_meta = pd.read_parquet('data/symbol_meta.parquet')

# cluster_coheasion = mc.get_cluster_coheasion(sim_df=resid_par_cor_mat.abs(), symbol_meta=symbol_meta, cluster_col='cluster_n200')

In [ ]:
symbol_meta['liquidy_rank'] = symbol_meta.groupby('cluster_n200')['daily_avg_dollar_volume'].rank(method='first')

symbol_meta = symbol_meta.sort_values(['cluster_n200', 'liquidy_rank']).reset_index()

In [ ]:
by_cluster = symbol_meta.groupby('cluster_n200')
result = []
for cluster, frame in by_cluster:
    counts = dict(frame['sector'].value_counts())
    counts.pop('', None)  # drop symbols w/o sector/industry info
    result.append(counts)

cluster_sec = pd.DataFrame(result)    
cluster_sec = cluster_sec.drop(columns=['Finance'])
cluster_sec_pct = cluster_sec.apply(lambda x: 100 * x / float(x.sum()), axis=1)

In [ ]:
cluster_sec_pct.style.background_gradient(cmap='coolwarm', axis=0)

In [ ]:
# n += 1
n = 61
print(n)
symbol_meta[symbol_meta.cluster_n200==n].sector.value_counts()

In [ ]:
sym_clust = symbol_meta[symbol_meta.cluster_n200==n]
sym_clust